# Google search try

In [ ]:
from googlesearch import search
import re
import pandas as pd

In [ ]:
companies = pd.read_csv('../data/processed/companies.csv')

In [ ]:
companies = companies['name'].tolist()

In [ ]:
# to search
found = set()

for idx,company in enumerate(companies):
    for j in search(company, tld="com"):
        print(idx)
        if re.search('wikipedia',j):
            found.add(company)
            break

In [ ]:
search

In [ ]:
import bs4 as bs
import urllib

comp = 'The Bank of East Asia Ltd'
url = 'https://en.wikipedia.org/wiki/'
print(urllib.parse.urlencode(comp))
# source = urllib.request.urlopen(url).read()

In [ ]:
soup = bs.BeautifulSoup(source,'html')

In [ ]:
for tag in soup.find_all('td'):
    if tag.get('class') == ['infobox-data', 'adr']:
        print(tag.text)

# Nominatim

In [34]:
from geopy.geocoders import Nominatim
import pandas as pd 
import os
import sys
import ssl
import numpy as np


sys.path.append('../')

In [35]:
ssl_context = ssl.create_default_context()
ssl_context.check_hostname = False
ssl_context.verify_mode = ssl.CERT_NONE

In [36]:
geolocator = Nominatim(user_agent="my-application3", ssl_context=ssl_context, timeout=None)

In [37]:
# Random test
location = geolocator.geocode("175 5th Avenue NYC, United states")
location.latitude 

40.741059199999995

In [38]:
address = pd.read_parquet('../data/raw/addresses.parquet.gzip')
country_codes  = pd.read_excel('../data/raw/Countries.xlsx')

In [41]:
address.loc[address['name']=='Himax Technologies, Inc.'] 

,id,name,country,city_state_postal,location_street1
190,64553,"Himax Technologies, Inc.",TWN,"Tainan, Tainan City 744 48",No. 15 Zih Lian Road


In [25]:
address['completed'] = np.nan 

In [26]:
address['city'] = address['city_state_postal'].str.split(',').str[0]

In [27]:
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Aachen'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Artarmon'), 'country'] = 'AUS'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Ann Arbor'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Bangalore'), 'country'] = 'IND'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Blindern'), 'country'] = 'NOR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Boulder'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Cambridge'), 'country'] = 'GBR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Lausanne'), 'country'] = 'CHE'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Milwaukee'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Munich'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'New York'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Norwich'), 'country'] = 'GBR'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Oberhaching'), 'country'] = 'DEU'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Paris'), 'country'] = 'FRA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Philadelphia'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Snoqualmie'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Southborough'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Stockholm 111 37'), 'country'] = 'SWE'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Towson'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Waltham'), 'country'] = 'USA'
address.loc[(address['country'] == 'NAM')&(address['city'] == 'Zurich 8002'), 'country'] = 'CHE'

# Adding the only other company with location_street1
address.loc[(address['name'] == 'Fennoa Oy'), 'city'] = 'Alavus'
address.loc[(address['name'] == 'Fennoa Oy'), 'country'] = 'FIN'
address.loc[(address['name'] == 'Fennoa Oy'), 'city_state_postal'] = 'Alavus, 63300'
address.loc[(address['name'] == 'Fennoa Oy'), 'location_street1'] = 'Mikko Kalliovaara Rinteentie 136'

In [28]:
address.loc[(address.country == 'NAM'), 'country'] = None

In [29]:
address = address.merge(country_codes, how='left', left_on='country', right_on='ISO-alpha3 Code')

In [30]:
# Have to take care of this when finding the location
# address.loc[(~address['location_street1'].isnull())&(address.city_state_postal.isnull())] 

In [31]:
address.drop(columns=['No','M49 Code','Region 1', 'Region 2', 'country'], inplace=True)
address.rename(columns={'Country or Area':'country'}, inplace=True)

In [32]:
address.head(2)

,id,name,city_state_postal,location_street1,completed,city,country,ISO-alpha3 Code,Continent
0,50472,"Infinity Pharmaceuticals, Inc.","Cambridge, MA 02138",1100 Massachusetts Avenue,NaN,Cambridge,United States of America,USA,North America
1,282940,"Bozhon Precision Industry Technology Co., Ltd.","Suzhou, Jiangsu 215222",666 Huxin West Road,NaN,Suzhou,China,CHN,Asia


In [33]:
address.loc[address.country.isnull()] 

,id,name,city_state_postal,location_street1,completed,city,country,ISO-alpha3 Code,Continent
190,64553,"Himax Technologies, Inc.","Tainan, Tainan City 744 48",No. 15 Zih Lian Road,NaN,Tainan,NaN,NaN,NaN
498,276151,"ASE Technology Holding Co., Ltd.",Kaohsiung City 811 70,26 Chin 3rd Road,NaN,Kaohsiung City 811 70,NaN,NaN,NaN
518,25172,"Chunghwa Telecom Co., Ltd.","Taipei, Taipei City 100 48",No. 21-3 Xinyi Road,NaN,Taipei,NaN,NaN,NaN
540,25816,Evergreen Marine Corp. (Taiwan) Ltd.,"Taipei, Taipei City 104","166, Section 2",NaN,Taipei,NaN,NaN,NaN
787,230636,"TBI MOTION Technology Co., Ltd.","New Taipei, Taipei City 236 80","No.123, Sanduo Road",NaN,New Taipei,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
195479,313941,"Qifeng Technology Co., Ltd.",None,None,NaN,None,NaN,NaN,NaN
195570,304573,Chargelogic Holdings Llc,None,None,NaN,None,NaN,NaN,NaN
195671,304704,Mantle Labs Ltd.,None,None,NaN,None,NaN,NaN,NaN
195833,243059,Ruiganglian Group Ltd.,None,None,NaN,None,NaN,NaN,NaN


### Getting the latitude and longitude from Nominatim

In [14]:
def api_call(search_str):
    """
    This function takes a search string and returns a location object obtained from the Nomnatim API.

    Parameters:
    search_str (str): The search string to be used in the API call.

    Returns:
    location (object): The location object returned from the API call. 
    """
    try:
        location = geolocator.geocode(search_str)
        return location
    except:
        return None



def new_search_str(old_search_str, city_state_postal, location_street1, country):
    """
    This function modifies the search string to be used in the API call. If the search string is None, it creates one from the 
    city_state_postal, location_street1, and country columns. If the search string is not None, it modifies the search string.

    Parameters:
    old_search_str (str): The search string to be modified.
    city_state_postal (str): The city, state, and postal code of the company.
    location_street1 (str): The street address of the company.
    country (str): The country of the company.

    Returns:
    search_str (str): The modified search string.

    """

    if old_search_str is None:

        # we create a new search string

        if (city_state_postal is None) & (location_street1 is None) & (country is None):
            search_str = None
        
        elif (city_state_postal is None) & (location_street1 is None) & (country is not None):
            search_str = country

        elif (city_state_postal is None) & (location_street1 is not None) & (country is None):
            search_str = location_street1

        elif (city_state_postal is None) & (location_street1 is not None) & (country is not None):
            search_str = location_street1 + ',' + country

        elif (city_state_postal is not None) & (location_street1 is None) & (country is None):
            search_str = city_state_postal

        elif (city_state_postal is not None) & (location_street1 is None) & (country is not None):
            search_str = city_state_postal + ',' + country
        
        elif (city_state_postal is not None) & (location_street1 is not None) & (country is None):
            search_str = city_state_postal + ',' + location_street1

        elif (city_state_postal is not None) & (location_street1 is not None) & (country is not None):
            search_str = city_state_postal + ',' + location_street1 + ',' + country
        


    else:


    return search_str


In [18]:
address_new = address.loc[(address['completed'] != 1)]

for id, rows in address_new.iterrows():

    print(id)

    if rows['completed'] != 1:

        if rows['city_state_postal'] is None:

            if rows['location_street1'] is None:

                if rows['country']:
                    search_str = rows['country']
                
                else:
                    address.loc[id,'completed'] = 1
                    continue
        
            else:
                search_str = rows['location_street1'] + ", " + rows['country']
        else:

            if rows['location_street1'] is None:

                search_str = rows['city_state_postal'] + ", " + rows['country']
               
            else:

                search_str = rows['location_street1'] + ", " + rows['city_state_postal'] + ", " + rows['country']
              

            
        x = api_call(search_str)
        if x is None:
            print(search_str)
            print('here!')
            address.loc[id,'completed'] = 0
            continue
        if x is not None:
            lat = x.latitude
            log = x.longitude
            address.loc[id,'lat'] = lat
            address.loc[id,'log'] = log
            address.loc[id,'completed'] = 1
            
            # i = 1
            # while i < len(rows['city_state_postal'].split(',')) :
            #     search_str = ','.join(rows['city_state_postal'].split(',')[0:i])+", " + rows['country']
            #     print(search_str)
            #     x = geolocator.geocode(search_str)
            #     if x is None:
            #         i += 1
            #         continue
            #     if x is not None:
            #         lat = x.latitude
            #         log = x.longitude
            #         address.loc[id,'lat'] = lat
            #         address.loc[id,'log'] = log
            #         address.loc[id,'completed'] = 1
            #         break
    
    if id % 100 == 0:
        address.to_parquet('../data/processed/addresses1.parquet.gzip', compression='gzip')
        print('saved')
        break

1
666 Huxin West Road, Suzhou, Jiangsu 215222, China
here!
2
One Quaker Park, Conshohocken, PA 19428-2380, United States of America
here!
3
N56 West17000 Ridgewood Drive, Menomonee Falls, WI 53051, United States of America
here!
5
R- 211, Mahalaxmi Nagar, Indore, Madhya Pradesh 452010, India
here!
6
10 Moti Kind Street, Rehovot, HaMerkaz 7638519, Israel
here!
7
Room 818, Yisibo Software Building, Shenzhen, Guangdong 518000, China
here!
9
10
11
12
13
14
15
3400 Cumberland Boulevard SE, Atlanta, GA 30339, United States of America
here!
16
17
12 Ha`Amal Street, Rosh HaAyin, HaMerkaz 4809246, Israel
here!
18
19
20
21
22
23
24
100 Vaughan Valley Boueivard, Vaughan, ON L4H 3C5, Canada
here!
25
26
Northern Industrial Zone, Yavne, HaMerkaz 81106, Israel
here!
27
28
29
30
Sumangal Builder House, Nashik, Maharashtra 422002, India
here!
31
32
33
34
1020 Stony Hill Road, Yardley, PA 19067, United States of America
here!
35
36
37
Alicia Moreau de Justo, No. 50, Buenos Aires C1107AAB, Argentina
here

In [ ]:
address = pd.read_parquet('../data/processed/addresses1.parquet.gzip')
address

In [ ]:
address = pd.read_parquet('../data/processed/addresses.parquet.gzip')
address

In [ ]:
address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')

In [ ]:
address.country.isna().sum()

In [ ]:
country_codes 

In [ ]:
uni = address.groupby('city').agg(
    unique_country = pd.NamedAgg('country',lambda x: x.unique().tolist())
).reset_index()

In [ ]:
mult = uni.loc[uni['unique_country'].apply(lambda x: len(x)) > 1]

In [ ]:
mult_city = mult.explode('unique_country')

In [ ]:
address = mult_city.merge(country_codes, left_on='unique_country', right_on='Alpha-3 code', how='left')

In [ ]:
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Aachen'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Artarmon'), 'Country'] = 'Australia'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Ann Arbor'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Bangalore'), 'Country'] = 'India'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Boulder'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Cambridge'), 'Country'] = 'United Kingdom'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Lausanne'), 'Country'] = 'Switzerland'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Milwaukee'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Munich'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'New York'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Norwich'), 'Country'] = 'United Kingdom'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Oberhaching'), 'Country'] = 'Germany'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Paris'), 'Country'] = 'France'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Philadelphia'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Snoqualmie'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Southborough'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Stockholm 111 37'), 'Country'] = 'Sweden'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Towson'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Waltham'), 'Country'] = 'United States'
address.loc[(address['unique_country'] == 'NAM')&(address['city'] == 'Zurich 8002'), 'Country'] = 'Switzerland'

In [ ]:
address

In [ ]:
for id, row in address.iterrows():
    print(id)
    mask = (address['city'] == row['city'])&(address['country'] == row['unique_country'])

    x = geolocator.geocode(row['city']+ ", "+ row['Country'])
    
    if x is not None:

        lat = x.latitude
        log = x.longitude

        address.loc[mask,'lat'] = lat
        address.loc[mask,'log'] = log

    if x is None:

        address.loc[mask, 'lat'] = None
        address.loc[mask, 'log'] = None
    
    if id % 100 == 0:
        address.to_parquet('../data/processed/addresses.parquet.gzip', compression='gzip')
        print('saved')
    

In [ ]:
address.to_csv('../data/processed/addresses.csv')